## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [2]:
# IMPORT PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [3]:
data_path = ''
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).

In [4]:
# display data
x

,AST,REB,STL
0,41,43,14
1,23,43,8
2,20,39,7
3,19,47,6
4,21,43,4
...,...,...,...
7375,17,39,10
7376,26,40,10
7377,23,52,8
7378,23,41,11


In [5]:
y

0       144
1        92
2        98
3       101
4       110
       ... 
7375     87
7376    107
7377    116
7378     95
7379     97
Name: PTS, Length: 7380, dtype: int64


#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [6]:
x['AST']

0       41
1       23
2       20
3       19
4       21
        ..
7375    17
7376    26
7377    23
7378    23
7379    17
Name: AST, Length: 7380, dtype: int64

In [7]:
x['STL']

0       14
1        8
2        7
3        6
4        4
        ..
7375    10
7376    10
7377     8
7378    11
7379     4
Name: STL, Length: 7380, dtype: int64

In [8]:
# x['AST']
x['AST'].mul(x['STL'], axis=0)

0       574
1       184
2       140
3       114
4        84
       ... 
7375    170
7376    260
7377    184
7378    253
7379     68
Length: 7380, dtype: int64

In [9]:
# call mentor about this challenge

def psi(x):
    # make a copy of df to return
    copy = x.copy()
    copy['1'] = np.ones(copy.shape[0])

    # add the new data to the existing df
    copy['AST^2'] = np.power(copy['AST'], 2)
    copy['REB^2'] = np.power(copy['REB'], 2)
    copy['STL^2'] = np.power(copy['STL'], 2)

    copy['ASTSTL'] = copy['AST'].mul(copy['STL'], axis=0)
    copy['REBSTL'] = copy['REB'].mul(copy['STL'], axis=0)
    copy['ASTREB'] = copy['AST'].mul(copy['REB'], axis=0)     
    return copy

#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

In [10]:
X = psi(x)

In [11]:
X.shape

(7380, 10)

#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

In [12]:
def p2(x,w):
    return x.dot(w)

In [13]:
p2(X, [0.06, 0.05, 0.03, 0.01, 0.02, 0.02, 0.04, 0.03, 0.02, 0.01])

0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

In [14]:
def Loss(x, y, w):
    return np.mean((p2(x,w) - y)**2)

#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

In [24]:
m = y.size

def my_descent(x, y, alpha, w, iter):
    # prep empty lists
    lst = []
    # step 1
    transformed = psi(x)
    # step 2
    lst.append(Loss(transformed,y,w))
    # repeat steps for max num iter
    for i in range(iter):
        pred = p2(transformed,w)
        # step 3
        for coef in range(len(w)):
            # step 4
            error = (pred - y).dot(transformed.iloc[:,coef])
            errors = error.sum()
            w[coef] = w[coef] - alpha * (1.0/m) * errors
        lst.append(Loss(transformed,y,w))
    return lst

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


In [25]:
w = [0.06, 0.05, 0.03, 0.01, 0.02, 0.02, 0.04, 0.03, 0.02, 0.01]
my_descent(x, y, 0.001, w, 100)

[891.9364108807587,
 12217158149.208988,
 3.4784046044396314e+17,
 9.903914693999095e+24,
 2.8198998995089725e+32,
 8.028982164122205e+39,
 2.2860582605438562e+47,
 6.508997359532879e+54,
 1.8532793917652294e+62,
 5.276764322098587e+69,
 1.5024308711732462e+77,
 4.277808112825974e+84,
 1.2180022789247905e+92,
 3.467966566845245e+99,
 9.874195078988864e+106,
 2.811437958775423e+114,
 8.004888836825293e+121,
 2.279198268982632e+129,
 6.489465194614301e+136,
 1.8477180807490043e+144,
 5.260929835574382e+151,
 1.4979223845456544e+159,
 4.264971288821542e+166,
 1.2143473041154545e+174,
 3.457559911077219e+181,
 9.844564646517067e+188,
 2.8030014105889876e+196,
 7.980867808657785e+203,
 2.272358862847882e+211,
 6.4699916417131315e+218,
 1.8421734580855275e+226,
 5.245142865093761e+233,
 1.4934274269609364e+241,
 4.2521729854908226e+248,
 1.2107032971352323e+256,
 3.4471844835468913e+263,
 9.815023128890636e+270,
 2.7945901787518173e+278,
 7.956918862664801e+285,
 2.265539980366962e+293,
 6.4

#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?

In [27]:
alphas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]
for alpha in alphas:
    print(my_descent(x, y, alpha, w, 100)[-1])

nan
nan
nan
nan
nan
nan
nan
nan
nan
